<a href="https://colab.research.google.com/github/Maxxx-VS/The-Founder/blob/master/Last_task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Задание:
Разработать нейросетевую архитектуру, которая сможет анализировать текстовые отзывы клиентов и определять их тональность: позитивная, негативная или нейтральная. Модель должна эффективно обрабатывать большое количество текстов и учитывать наличие возможного дисбаланса в данных (например, больше позитивных отзывов, чем негативных).

# Подготовка данных

1. Загрузка и первичный анализ:
- Импортируем данные из CSV/JSON
- Проводим анализ распределения классов: считаем количество примеров для каждой категории (позитив/негатив/нейтраль)
- Визуализируем дисбаланс через гистограмму

2. Текстовая предобработка:
- Приводим текст к нижнему регистру
- Удаляем спецсимволы, цифры и лишние пробелы
- Обрабатываем сокращения и сленг
- Лемматизация/стемминг с использованием nltk
- Удаление стоп-слов (с корректировкой списка под контекст отзывов)

# Борьба с дисбалансом классов

1. Методы на уровне данных:
- Стратифицированное разбиение на train/val/test (сохранение пропорций)
- Применение SMOTE для генерации синтетических примеров минорных классов
2. Методы на уровне модели:
- Расчет весов классов через sklearn.utils.class_weight.compute_class_weight
- Использование взвешенной функции потерь (weighted_categorical_crossentropy)
- Настройка метрик качества с акцентом на F1-score вместо accuracy

#  Векторизация текста

1. Tokenization:
- Создание словаря (10,000-50,000 наиболее частых слов)
- Замена редких слов на специальный токен <UNK>
- Преобразование текста в последовательности индексов

2. Padding:
- Обрезка/дополнение последовательностей до фиксированной длины (напр., 200 токенов)
- Использование пост-обрезки (truncating='post') для сохранения концов текстов
3. Дополнительные методы:
- Использование предобученных эмбеддингов (GloVe, FastText)
- Динамическое обучение векторных представлений

#  Модель

- Выбор подходящей модель
- выбор количества скрытых слоев и общей емкости сети
- Регуляризация
- Выбор функций активации

#  Обучение

1. Параметры обучения:
- Оптимизатор: AdamW (с L2-регуляризацией)
- Learning rate: 3e-4 с расписанием ReduceLROnPlateau
- Размер батча: 64-128 (зависит от объема данных)
- Ранняя остановка (EarlyStopping) при застое val_loss

2. Стратегии:
- Микс-ап (Mixup) для аугментации текстовых данных
- Перекрестная проверка (3-5 folds)
- Мониторинг метрик для каждого класса отдельно

#  Оценка результатов

1. Метрики:
- Матрица ошибок с акцентом на recall минорных классов
- Отчет классификации (precision/recall/F1 по каждому классу)
- ROC-AUC для многоклассового случая

2. Интерпретация:
- Анализ примеров с ошибками
- Визуализация эмбеддингов через UMAP/t-SNE
- LIME/SHAP для объяснения предсказаний

In [1]:
# Пример кода
# 1. Импорт библиотек
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report

# 2. Загрузка и предобработка данных
# Предположим, данные в CSV с колонками 'text' и 'sentiment'
df = pd.read_csv('reviews.csv')
texts = df['text'].values
labels = df['sentiment'].values

# Преобразование текстов в последовательности
tokenizer = Tokenizer(num_words=10000, oov_token='<OOV>')
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
padded_sequences = pad_sequences(sequences, maxlen=200, truncating='post')

# Преобразование меток в числовой формат
label_map = {'positive': 0, 'negative': 1, 'neutral': 2}
labels = np.array([label_map[label] for label in labels])

# 3. Обработка дисбаланса классов
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(labels),
    y=labels
)
class_weights = dict(enumerate(class_weights))

# 4. Создание модели
model = Sequential([
    Embedding(input_dim=10000, output_dim=128, input_length=200),
    Bidirectional(LSTM(64, return_sequences=True)),
    Dropout(0.5),
    Bidirectional(LSTM(32)),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(3, activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# 5. Обучение с учетом весов классов
X_train, X_test, y_train, y_test = train_test_split(
    padded_sequences, labels, test_size=0.2, stratify=labels
)

history = model.fit(
    X_train,
    y_train,
    epochs=10,
    validation_split=0.1,
    class_weight=class_weights,
    batch_size=64
)

# 6. Оценка модели
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)

print(classification_report(y_test, y_pred_classes))